In [14]:
import json

In [15]:
with open("datasets_study_coderevaljava.json", "r") as f:
    data = json.load(f) 

analysis_data = []
prompt_data = []    

dataset_name = [ ]
for item in data:
    if item['model']=='datasets_study.analysis':
        analysis_data.append(item)
    elif item['model']=='datasets_study.prompt':
        prompt_data.append(item)
        dataset_name.append(item['fields']['source_dataset'])



len(analysis_data),len(prompt_data)
len(set(dataset_name))


20

In [16]:
problem_list = {}
for item in data:
    if item['model']=='datasets_study.problem':
       problem_list[item['pk']] = item['fields']['description']
print(problem_list)

{1: 'Spelling and grammatical errors', 2: 'Very short sentence.', 3: 'Partial or incomplete sentence.', 4: 'Self-Admitted Technical Debt (ex: TODO / FIXME / hack / XXX, etc)', 5: 'Automatically generated code/comments (ex:: // TODO auto-generated method stub).', 6: "Method name mismatches with the function/method's intent.", 7: 'Not using standard JavaDoc (Java) or docstring (Python)', 8: 'Inconsistent prompt style', 9: 'URL or reference in the comment.', 12: 'Check whether the question mark is indeed asking a question about what the function does instead of being just part of the description of the code.', 13: 'Check the JavaDoc quality.', 14: 'Comment includes PII(author name, email address, etc.)', 15: 'The prompt description is unclear.', 16: 'Incorrect input/output pair example.'}


In [17]:
from collections import defaultdict
users = defaultdict(list)
analysis = defaultdict(list)
problems = defaultdict(list)


for item in analysis_data:
    # if item['fields']['user_id'] != 4:
    #     continue
    
    users[item['fields']['user_id']].append(item['fields'])
    analysis[item['fields']['prompt']].append(item['fields'])
    for problem in item['fields']['problems']:
        problems[problem].append(item['fields'])
print(users.keys())
print(len(analysis))
print(len(problems))    

dict_keys([4, 3, 1, 11])
4164
12


In [18]:
problems = sorted(problems.items(), key=lambda x: x[0])
print(problems)

[(1, [{'user_id': 4, 'prompt': 3, 'comments': None, 'problems': [1, 8]}, {'user_id': 4, 'prompt': 4, 'comments': None, 'problems': [1]}, {'user_id': 4, 'prompt': 5, 'comments': None, 'problems': [1, 8]}, {'user_id': 4, 'prompt': 22, 'comments': None, 'problems': [1]}, {'user_id': 4, 'prompt': 28, 'comments': None, 'problems': [1, 7, 8]}, {'user_id': 4, 'prompt': 29, 'comments': None, 'problems': [1, 8]}, {'user_id': 4, 'prompt': 33, 'comments': None, 'problems': [1, 8]}, {'user_id': 4, 'prompt': 39, 'comments': None, 'problems': [1, 8]}, {'user_id': 4, 'prompt': 40, 'comments': None, 'problems': [1, 8]}, {'user_id': 4, 'prompt': 48, 'comments': None, 'problems': [1, 8]}, {'user_id': 4, 'prompt': 50, 'comments': None, 'problems': [1, 8]}, {'user_id': 4, 'prompt': 55, 'comments': None, 'problems': [1, 6, 8]}, {'user_id': 4, 'prompt': 64, 'comments': None, 'problems': [1, 8]}, {'user_id': 4, 'prompt': 65, 'comments': None, 'problems': [1, 8]}, {'user_id': 3, 'prompt': 71, 'comments': None

In [19]:
for item in problems:
    key = item[0]
    total = len(item[1])
    print(f"{key}. {problem_list[key]}: {total}")


1. Spelling and grammatical errors: 1122
2. Very short sentence.: 1
3. Partial or incomplete sentence.: 339
4. Self-Admitted Technical Debt (ex: TODO / FIXME / hack / XXX, etc): 1
6. Method name mismatches with the function/method's intent.: 3961
7. Not using standard JavaDoc (Java) or docstring (Python): 1313
8. Inconsistent prompt style: 911
9. URL or reference in the comment.: 54
12. Check whether the question mark is indeed asking a question about what the function does instead of being just part of the description of the code.: 444
13. Check the JavaDoc quality.: 3040
15. The prompt description is unclear.: 326
16. Incorrect input/output pair example.: 57


In [20]:
merged_analysis = defaultdict(list)
for item in analysis_data:
    if item['fields']['user_id'] != 3:
        continue
    merged_analysis[item['fields']['prompt']].append(item['fields'])


In [21]:
user_3_analysis = defaultdict(str)
user_4_analysis = defaultdict(str)
for item in users[3]:
    item['problems'] = '#'.join(sorted( [str(x) for x in item['problems']] ))
    user_3_analysis[item['prompt']] = item['problems']


for item in users[4]:
    item['problems'] = '#'.join(sorted( [str(x) for x in item['problems']] ))
    user_4_analysis[item['prompt']] = item['problems']

len(user_3_analysis), len(user_4_analysis)

(3566, 3566)

In [22]:
user_3 = []
user_4 = []
problem_wise = defaultdict(list)
for key in merged_analysis.keys():
    if key not in user_3_analysis:
        user_3_analysis[key] = ''
    if key not in user_4_analysis:
        user_4_analysis[key] = ''
    user_3.append(user_3_analysis[key])
    user_4.append(user_4_analysis[key])

    for i in range(1,17):
        if str(i) in user_3_analysis[key].split('#'):
            problem_wise[i].append(1)
        else:
            problem_wise[i].append(0)
            
        if str(i) in user_4_analysis[key].split('#'):
            problem_wise[i].append(1)
        else:
            problem_wise[i].append(0)


In [23]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(user_3, user_4)

0.7557499003181963

In [24]:
for i in range(1,17):
    user_3 = []
    user_4 = []
    # print(problem_wise[i])
    for index, item in enumerate(problem_wise[i]):
        if index%2==0:
            user_3.append(item)
        else:
            user_4.append(item)
    print(sum(user_3), sum(user_4))
    print(f"Problem {i}: {cohen_kappa_score(user_3, user_4)}")

472 347
Problem 1: 0.4182699019585914
1 0
Problem 2: 0.0
84 123
Problem 3: 0.6868869320890093
1 0
Problem 4: 0.0
0 0
Problem 5: nan
1317 1323
Problem 6: 0.9615101413990959
435 439
Problem 7: 0.9869604359295205
297 302
Problem 8: 0.8450869498110543
18 18
Problem 9: 1.0
0 0
Problem 10: nan
0 0
Problem 11: nan
147 145
Problem 12: 0.9357247141618318
1016 1007
Problem 13: 0.9924095271599448
0 0
Problem 14: nan
119 83
Problem 15: 0.2873884135670832
5 37
Problem 16: -0.002476539805357225


/Users/lsiddiqsunny/.pyenv/versions/3.9.4/lib/python3.9/site-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/Users/lsiddiqsunny/.pyenv/versions/3.9.4/lib/python3.9/site-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/Users/lsiddiqsunny/.pyenv/versions/3.9.4/lib/python3.9/site-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/Users/lsiddiqsunny/.pyenv/versions/3.9.4/lib/python3.9/site-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
